In [1]:
! pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25858 sha256=1d961ec32afd98989e657d1cbde5fd9fea1d55d632c5a33c8c6c15c4ab9d8610
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.models import load_model
# from keras.layers import LeakyReLU
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [12]:
print("[INFO] loading images...")
imagePaths = list(paths.list_images("../input/human-anime/human_anime"))
# print(imagePaths)
data = []
labels = []

[INFO] loading images...


In [13]:
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the input image (224x224) and preprocess it
	image = load_img(imagePath, target_size=(72, 72))
	image = img_to_array(image)
# 	image = preprocess_input(image)
	# update the data and labels lists, respectively
	# count+=1/
	# if count%10==0:
	data.append(image)
	labels.append(label)
    


In [14]:
data = np.array(data, dtype="float32")
labels1 = np.array(labels)
del labels
print("labels",labels1.shape)
# perform one-hot encoding on the labels
lb = LabelEncoder()
labels = lb.fit_transform(labels1)
labels = to_categorical(labels)
print(labels,type(labels))

labels (37369,)
[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] <class 'numpy.ndarray'>


In [15]:
print(data.shape,labels.shape)
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.2, stratify=labels, random_state=42)
print(trainX.shape, testX.shape, trainY.shape, testY.shape)

(37369, 72, 72, 3) (37369, 7)
(29895, 72, 72, 3) (7474, 72, 72, 3) (29895, 7) (7474, 7)


In [16]:
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer,Conv2D,MaxPool2D,BatchNormalization
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model

In [23]:
INIT_LR = 1e-3
EPOCHS = 200
BS = 256


baseModel = Sequential()
# baseModel.add(Dense(12, activation='relu', input_tensor = Input(shape=(48,48,3))))
# baseModel.add(InputLayer(input_shape = (48,48,3)))
baseModel = tf.keras.applications.VGG19(input_shape=(72,72,3),include_top=False,weights='imagenet')
headModel = baseModel.output

# headModel = Conv2D(48,3,activation="relu",padding = "same")(headModel)
# # headModel = Conv2D(48,3,activation="relu",padding = "same")(headModel)
# headModel = MaxPool2D(pool_size = (2,2))(headModel)
# headModel = BatchNormalization()(headModel)
# # headModel = Dropout(0.2)(headModel)
# headModel = Conv2D(96,3,activation="relu",padding = "same")(headModel)
# headModel = Conv2D(96,3,activation="relu",padding = "same")(headModel)
# headModel = MaxPool2D(pool_size = (2,2))(headModel)
# headModel = BatchNormalization()(headModel)
# headModel = Dropout(0.15)(headModel)

# headModel = Conv2D(192,3,activation="relu",padding = "same")(headModel)
# headModel = Conv2D(192,3,activation="relu",padding = "same")(headModel)
# headModel = Conv2D(192,3,activation="relu",padding = "same")(headModel)
# headModel = MaxPool2D(pool_size = (2,2))(headModel)
# headModel = BatchNormalization()(headModel)
# # headModel = Dropout(0.2)(headModel)

# headModel = Conv2D(384,3,activation="relu",padding = "same")(headModel)
# headModel = Conv2D(384,3,activation="relu",padding = "same")(headModel)
# headModel = Conv2D(384,3,activation="relu",padding = "same")(headModel)
# headModel = MaxPool2D(pool_size = (4,4))(headModel)
# # headModel = BatchNormalization()(headModel)
# # headModel = Dropout(0.2)(headModel)


# headModel = Conv2D(384,3,activation="relu",padding = "same")(headModel)
# # headModel = BatchNormalization()(headModel)
# headModel = Conv2D(384,3,activation="relu",padding = "same")(headModel)
# # headModel = BatchNormalization()(headModel)
# headModel = Conv2D(384,3,activation="relu",padding = "same")(headModel)
# # headModel = BatchNormalization()(headModel)
# headModel = MaxPool2D(pool_size = (2,2),padding = "valid")(headModel)
# headModel = Dropout(0.2)(headModel)

# construct the head of the model that will be placed on top of the
# the base model
# headModel = baseModel.output
# headModel = AveragePooling2D(pool_size=(1, 1))(headModel)
headModel = Flatten(name="flatten")(headModel)
# headModel = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.001))(headModel)
# headModel = Dropout(0.15)(headModel)
headModel = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.001))(headModel)
headModel = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.001))(headModel)
headModel = Dropout(0.15)(headModel)
headModel = Dense(156, activation="relu",kernel_regularizer=regularizers.l2(0.001))(headModel)
# headModel = Dropout(0.1)(headModel)
# headModel = Dense(256, activation="relu",kernel_initializer = 'he_normal')(headModel)
# headModel = Dense(1024, activation="relu")(headModel)
# headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(7, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)


checkpoint = ModelCheckpoint('EmotionDetectionModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=0)

# earlystop = EarlyStopping(monitor='val_loss',
#                           min_delta=0,
#                           patience=3,
#                           verbose=1,
#                           restore_best_weights=True
#                           )

# reduce_lr = ReduceLROnPlateau(monitor='val_loss',
#                               factor=0.1,
#                               patience=5,
#                               verbose=1,
#                               min_delta=0.0001)

callbacks = [checkpoint]

for layer in baseModel.layers:
	layer.trainable = False

print(model.summary())

80142336/80134624 [==============================] - 2s 0us/step
Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 72, 72, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 72, 72, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 72, 72, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 36, 36, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 36, 36, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 36, 36, 128)       147584    
_______________________________________________________

In [25]:
# for layer in baseModel.layers:
# 	layer.trainable = False

# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS,verbose=1)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])



[INFO] compiling model...


In [ ]:
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
    callbacks=callbacks,
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

model = load_model("EmotionDetectionModel.h5")


# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
print(predIdxs)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
print(predIdxs)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("/kaggle/working/model.h5", save_format="h5")



[INFO] training head...
Epoch 1/200
116/116 [==============================] - 46s 394ms/step - loss: 3.9752 - accuracy: 0.2769 - val_loss: 2.9633 - val_accuracy: 0.3543
Epoch 2/200
116/116 [==============================] - 44s 379ms/step - loss: 2.8584 - accuracy: 0.3499 - val_loss: 2.6699 - val_accuracy: 0.3883
Epoch 3/200
116/116 [==============================] - 44s 383ms/step - loss: 2.6091 - accuracy: 0.3753 - val_loss: 2.4637 - val_accuracy: 0.4017
Epoch 4/200
116/116 [==============================] - 44s 381ms/step - loss: 2.4271 - accuracy: 0.3854 - val_loss: 2.3024 - val_accuracy: 0.4154
Epoch 5/200
116/116 [==============================] - 45s 384ms/step - loss: 2.2779 - accuracy: 0.4019 - val_loss: 2.1717 - val_accuracy: 0.4209
Epoch 6/200
116/116 [==============================] - 43s 374ms/step - loss: 2.1726 - accuracy: 0.4026 - val_loss: 2.0882 - val_accuracy: 0.4268
Epoch 7/200
116/116 [==============================] - 44s 380ms/step - loss: 2.0857 - accuracy: 0.4

116/116 [==============================] - 43s 367ms/step - loss: 1.4441 - accuracy: 0.4818 - val_loss: 1.4775 - val_accuracy: 0.4779
Epoch 56/200
116/116 [==============================] - ETA: 0s - loss: 1.4451 - accuracy: 0.4846
Epoch 00056: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
116/116 [==============================] - 44s 377ms/step - loss: 1.4451 - accuracy: 0.4846 - val_loss: 1.4632 - val_accuracy: 0.4744
Epoch 57/200
116/116 [==============================] - 44s 383ms/step - loss: 1.4212 - accuracy: 0.4899 - val_loss: 1.4578 - val_accuracy: 0.4798
Epoch 58/200
116/116 [==============================] - 44s 379ms/step - loss: 1.4249 - accuracy: 0.4918 - val_loss: 1.4558 - val_accuracy: 0.4802
Epoch 59/200
116/116 [==============================] - 43s 368ms/step - loss: 1.4249 - accuracy: 0.4891 - val_loss: 1.4590 - val_accuracy: 0.4813
Epoch 60/200
116/116 [==============================] - 44s 380ms/step - loss: 1.4275 - accuracy: 0.4865 - val_l

In [ ]:
history_dict = H.history
print(history_dict.keys())

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy for VGG19")
plt.xlabel(f"Epoch {EPOCHS}")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("/kaggle/working/plot")